# Exploring VCF Storage Solutions
# Notebook 1. Introduction to the project and preparation of example data
2025-02-12 Daniel P. Brink

# 1. Introduction

## 1.1. Background: 

The Jupyter notebooks collected in this in this GitHub repository - [titled Exploring VCF Storage Solutions](https://github.com/brinkdp/exploring-zarr-for-VCFs/tree/main/notebooks) -  are indended to document an investigation into a handful of software solutions designed to improve various aspects of handling genetic variant data stored in the VCF (Variant Call Format). The phrase _VCF Storage_ in the title is intended as a wider concept that just data storage: a common denominator among the tools that are investigated here are that VCF files are ingested in a new data storage format, queried and transformed with commands within and/or adjacent to the tools, saved to new version that will need to be tracked with metadata, and then possibly exported back to VCF in case compatibility with downstream bioinformatic pipelines is needed. Thus, these tools are not database management systems in the traditional sense, nor are they stricly computational tools. The phrase _VCF Storage_ might fail to communicate the width of use-cases that these tools could encompass, but it will do for now _in lieu_ of better descriptor.

The VCF format originated in during the [1000 Genomes project](https://www.internationalgenome.org/1000-genomes-summary) along with other seminal bioinformatics tools, utilities, and formats like SAMtools, HTSlib, and the SAM/BAM formats. The VCF format is a row-oriented tabular text file. While this makes VCFs quite human-readable and fairly easy to process with custom scripts, it is considered to be inefficient in terms of performance and storage ([Danechek et al., 2021](https://academic.oup.com/gigascience/article/10/2/giab008/6137722)). To try to improve processing times, a binary VCF version called BCF (binary VCF) has been developed. The specification for the VCF and BCF formats [can be found here](https://samtools.github.io/hts-specs/VCFv4.5.pdf) (v4.5, the latest version at the time of writing). 

Today, the gold standard tool for working with genetic variant data is `bcftools` ([see the manual here](https://samtools.github.io/bcftools/bcftools.html)). It supports a plethora of operations on data stored in compressed and uncompressed versions of the VCF and the BCF formats, from variant calling to data analysis operations. It is considered to be a highly optimized tool as far as VCF handling goes; in fact, it is probably more than sufficient for most users, although processing times can be quite substantial for large multi-sample datasets. 

So why investigate alternative tools at all, if `bcftools` is so efficient? The short answer is not VCF is not well suited at scale. Technological improvements in nucleotide sequencing - such as increased throughput, data quality, and read length - have been substantial since the first developments of the VCF format in the late 2000s, and data is, as a result, being generated at a higher rate and volume than before. Consequently, genetic variant data will also scale, adding more and more sequenced individuals to the populations described by the VCF files. It is thus likely that at some point in time - in the close future?, in the a distant future?, or, perhaps, in some cases already today? - the current way of working with the VCF format will pose a bottleneck in scientific analysis.

It should thus not come as a surprise that there are people that are working on adressing this by developing new technological solutions. As of the time of writing there are already a few available tools that claim to adress this problem. The idea behind this collection of notebooks is to investigate how they work, their benefits, drawbacks, and limitations. The focus will be on open-source solutions that can be tested without the need to pay for a licence. However, some of the tools that will be investigated are built around hybrid models where the core functionalities are available in open source releases and advanced features are offered for paying customers. Any new solution for handling VCF files will inevitably be compared to the standard operations of `bcftools`. Ideally, the candidate tools should be able to perform the same operations, and preferrably do so more efficiently.

"But what do you mean with ´more effciently´?", you might ask. In general, this investigation will consider efficiency in terms of three key paramaters:

- Processing performance
    - How long time and how much compute resources is required to perform standard VCF operations?
- Storage performance
    - How much disk space is required for storing the data. Is there efficient compression in place? How does the I/O of the software perform in a cloud storage enviroment.
- Data management
    - How can the software help the user to keep track of metadata and versioning?

The tools that will be investigated are:

- VCF Zarr
- TileDB-VCF
- OpenGCA

(more tools might be considered at a later stage.)

## 1.2. Outline of the notebooks

While these notebooks are essentially a vector to document a technical assessment of these tools, they have been writen in the ethos of Open Science and reproducible research. They serve both to document the findings of the investigations but also capture the process of work itself. The latter essentially leads to a much more verbose documentation that might be less efficient for readers that quickly want to find a particular piece of information, but hopefully valuable for readers intrested in reproducibility and learning about the tools. A "semi-naïve" approach was used for the tests cover the notebooks: with this I mean that part of the investigation was peformed by going in blind and discovering features, quirks, and errors organically, while other parts were based on reading documentation and tutorials before doing the tests. By working in this manner, the idea was to avoid reproducing existing documentation that the respective developers already provide and maintain, and instead focus on the experiences gained from interacting with the documentation and the tools.

One way to think about the individual notebooks is as chapters in a book. A rough table-of-contents for the planned notebooks looks something like this:

- Notebook 1. Introduction to the project and preparation of example data
- Notebook 2. Overview of commonly used bcftools operations (point-of-reference for all the comparisons)
- Notebook 3.1. VCF Zarr Part I
- Notebook 3.2. VCF Zarr Part II
- Notebook 4.1. TileDB-VCF Part I
- Notebook 4.1. TileDB-VCF Part II
- Notebook 5.1. OpenGCA Part I
- Notebook 5.2. OpenGCA Part II

The first two notebooks set the scope and serve as point-of-reference. New readers are reccomended to start there. Through the use of conda enviroments, the intention is that reader should be able to re-run the code on their own machines, reproducing the results and output files. Section 2 in each notebook will specify the installations commands that were used to setup each environment. Since the evaluated software have different dependencies, we will in general make one enviroment per evaluated tool.

The notebooks for the tools use subnumbering to be able to split the content into smaller chunks and to allow to add more notebooks for a tools over time without having to retroactively have to change the numbering. Notebooks that start with 3 are about VCF Zarr, 4 are about TileDB-VCF, and 5 are about OpenGCA. The notebooks covering the tools are meant to be read independently of each other, but their subnumbered notebooks are written to be read in order, starting with notebook X.1 (e.g. 3.1 -> 3.2 -> 3.3. etc.). In general, the order of the tests are preserved across the notebooks, meaning that a test performed for VCF Zarr in 3.1 is likely to have its TileDB-VCF equivalent reported in 4.1., and so on.

## 1.3. Preparing an example dataset for use across all notebooks
In order to make the exploration of these tools as systematic and comparable as possible, we will first start by preparing an example small dataset that will be used for testing all software. We will be using public human VCF data from the 1000 Genomes project (Section 3 in the current notebook). This is considered a large dataset, encompassing millions of variants across ~2500 samples. Using this data could eventually help us evaluate the performance of the software candidates at scale, should we need and desire to do so. However, for the intial operations, this data will take unneccessarily long to process, so we will downsample it in a reproducible manner to generate a small-to-medium sized dataset for testing out the basic operations.


# 2. Setup

All investigations in these notebooks will be done on a Mac M3 laptop. Conda environments will be used for reproducibility. Since there might be conflicting dependencies between the different tools that will be explored in these notebooks, we might end up using different environments for each notebook. The environments will be specified in the start of each notebook, as below. For the first two notebooks we will basically just need python and `bcftools` so the environment will be very slim.

```bash
CONDA_SUBDIR=osx-64 conda create -n explore_vcf_storage_solutions
conda activate explore_vcf_storage_solutions
conda install mamba -y
mamba install jupyter -y
mamba install bcftools -y 
pip install humanfriendly
```


`CONDA_SUBDIR=osx-64 conda` is used to ensure compatibility with the CPU architechture of the M series Macs. It essentially emulates an AMD64 chipset. Like all emulation, it comes at a cost of processing overhead, but for the needs of these notebooks we can live with that. But keep in mind that these notebooks are not focusing on optimizing the performance.


To test that the installation works, we can run:

In [1]:
import sys
import os
import requests
import humanfriendly

#Check that Conda and the libraries are installed as expected:
print(f"Current Conda environment: {os.environ['CONDA_DEFAULT_ENV']}")
print(f"Current Python version: {sys.version}")
!bcftools --version


Current Conda environment: explore_vcf_storage_solutions
Current Python version: 3.13.1 | packaged by conda-forge | (main, Jan 13 2025, 09:48:16) [Clang 18.1.8 ]
bcftools 1.21
Using htslib 1.21
Copyright (C) 2024 Genome Research Ltd.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.


# 3. Preparing the example data

Let's start with a small VCF dataset. A famous public dataset of genetic variants is the [1000 Genomes human VCF data](https://www.internationalgenome.org/1000-genomes-summary). Files for the lastest release ("Phase3") are [available here](https://hgdownload.cse.ucsc.edu/gbdb/hg19/1000Genomes/phase3/). This data is likely too large for the simple test of functionality that we will be doing in this notebook and we will want to downsample it to speed up calculations. Nevertheless, the full version of this dataset will be useful for later investigations in other notebooks.

Inspired by the [sgkit GWAS tutorial](https://github.com/sgkit-dev/sgkit/blob/7094d3cf192dfc25ff69456ec7f1e71e7df2c264/docs/examples/gwas_tutorial.ipynb), we can handle the downloaded of the VCF file the with the `requests` library if we want to keep the processing in python. (An alternative would be to use `curl`)

A function that download the example data can look something like:

In [2]:
def download_vcf(url, output_file):
    response = requests.get(url)
    if response.status_code == 200:
        # The response object contains binary, thus open the output file in write-binary mode
        with open(output_file, 'wb') as file:
            #Use chunks to speed up download. Use the default value of 8192 (2^13) bytes 
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"VCF file downloaded successfully and saved to {output_file}")
    else:
        print(f"Failed to download VCF file. HTTP status code: {response.status_code}")

For now, let's download the data for chromosome 1:

In [3]:
%%time
url = "https://hgdownload.cse.ucsc.edu/gbdb/hg19/1000Genomes/phase3/ALL.chr1.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz"
downloaded_file = "./input_data_temp/" + os.path.basename(url)
if not os.path.isfile(downloaded_file):
    download_vcf(url, downloaded_file)

VCF file downloaded successfully and saved to ./input_data_temp/ALL.chr1.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz
CPU times: user 6.85 s, sys: 5.32 s, total: 12.2 s
Wall time: 60 s


Despite only spanning one single chromosome, this file is still quite large for our current purposes:

In [4]:
file_size = os.path.getsize(downloaded_file)
print(f"{os.path.basename(downloaded_file)}\t{humanfriendly.format_size(file_size, binary=True)}")

ALL.chr1.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz	1.13 GiB


(The `binary=True` option is used to get file sizes in the same format as the `ls -lh` bash command for ease of comparison)

To get a feeling for the number of variants and samples in this file, we can use the gold standard VCF parsing software `bcftools`. For speeding up access to the file, we should first index the VCF. The default settings generates a .csi (coordinate-sorted index), but it is also possible generate tabix indexes. 

In [5]:
%%time
!bcftools index {downloaded_file}

CPU times: user 246 ms, sys: 105 ms, total: 351 ms
Wall time: 1min 2s


Let's count the number of variants and samples. By timing the command, we can get a feeling that even with this highly optimized tool, the parsing of this file takes quite some time! The number of variants are:

In [6]:
%%time
!bcftools view --no-header {downloaded_file} | wc -l

6468094
CPU times: user 1.81 s, sys: 596 ms, total: 2.4 s
Wall time: 6min 26s


and the number of samples contained in the file are:

In [7]:
%%time
!bcftools query -l {downloaded_file} | wc -l

2504
CPU times: user 2.66 ms, sys: 13.7 ms, total: 16.3 ms
Wall time: 177 ms


The downloaded file is a gzip compressed VCF. `bcftools` also supports the BCF (binary VCF) format. BCF is developed to for improved performance and compression. For the sake of faster operations during the preparation of the example data, we can afford to convert the file to BCF. Having access to the example data in both VCF and BCF format will also be useful for testing of ingestion operations of different VCF storage tools.

Conversion from VCF to BCF is done by setting the `-O` flag to `b`(compressed BCF). For piping of `bcftools` commands, it is reccomended to use the uncompressed BCF format `-Ou` to avoid losing processing time on compression/uncompression operations, but since we are not using a pipe here, we can make the compressed BCF directly.

In [8]:
%%time
!bcftools view {downloaded_file} -Ob -o ./input_data_temp/ALL.chr1.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.bcf

CPU times: user 1.14 s, sys: 485 ms, total: 1.63 s
Wall time: 5min 37s


Indexing of the BCF file is done with same command and will produce a `.csi` index.

In [9]:
%%time
!bcftools index {downloaded_file_bcf}

index: failed to open {downloaded_file_bcf}
CPU times: user 4.4 ms, sys: 18.6 ms, total: 23 ms
Wall time: 222 ms


Were there any gains in file size when using the compressed BCF?

In [10]:
downloaded_file_bcf="./input_data_temp/ALL.chr1.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.bcf"
file_size = os.path.getsize(downloaded_file_bcf)
print(f"{os.path.basename(downloaded_file_bcf)}\t{humanfriendly.format_size(file_size, binary=True)}")

ALL.chr1.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.bcf	971.47 MiB


It is a tad bit smaller, as expected. The original `.vcf.gz` was 1.2G, for comparison.

For illustration purposes, an example of an operation that is faster on the BCF is the command to count number of variants. For comparison, this took 6min 26s on the VCF.GZ version of the file. Using the BCF is faster for this operation.

In [11]:
%%time
!bcftools view --no-header {downloaded_file_bcf} | wc -l

6468094
CPU times: user 679 ms, sys: 284 ms, total: 963 ms
Wall time: 3min 9s


For our current purposes, it is great that it is smaller, but we still want to downsample the file. First, let's make a decision to only include the first 200 samples. `bcftools query -l` produces a list of all samples in the VCF/BCF file, from which we can extract a list with the first 200 samples that can be extracted with `bcftools view -S` 

In [18]:
%%time
downloaded_file_bcf_first_200_samples = "./input_data_temp/1kG_p3_chr1_first_200_samples.bcf"
!bcftools query -l {downloaded_file_bcf} > ./input_data_temp/1kG_p3_chr1_samples.txt
!head -n 200 ./input_data_temp/1kG_p3_chr1_samples.txt > ./input_data_temp/1kG_p3_chr1_samples_first_200_samples.txt
!bcftools view -S ./input_data_temp/1kG_p3_chr1_samples_first_200_samples.txt -Ob {downloaded_file_bcf} -o {downloaded_file_bcf_first_200_samples}

CPU times: user 123 ms, sys: 108 ms, total: 231 ms
Wall time: 42 s


Pretty fast! (As a footnote, the time it took to perform this operation on the original .vcf.gz file on the same laptop was circa 4 minutes. The BCF format sure has performance benefits. Those results are not included here since the focus is on the downsampling, not on benchmarking `bcftools`.)

Dropping these ~2000 samples gave a file size of:

In [24]:
file_size = os.path.getsize(downloaded_file_bcf_first_200_samples)
print(f"{os.path.basename(downloaded_file_bcf_first_200_samples)}\t{humanfriendly.format_size(file_size, binary=True)}")

1kG_p3_chr1_first_200_samples.bcf	226.2 MiB


The downsampling is getting somewhere! 

However, after we now have removed more than 2000 samples from the file, there is likely to be some variants in the dataset that have no ALT alleles in our subset of 200 samples. The AC field is used to quantify this: AC=0 means that all alleles from the samples have the reference allele, and a higher value, AC>=1 means that there is >=1 non-ref alleles.

To check the number variants in the 200 sample file that only contain the REF allele, we can set a filter on a maximum allowed AC with the flag `-C0`, where 0 is the max number.

In [25]:
%%time
!bcftools view --no-header -C0 ./input_data_temp/1kG_p3_chr1_first_200_samples.bcf |wc -l

5141248
CPU times: user 56.3 ms, sys: 41.2 ms, total: 97.5 ms
Wall time: 16.6 s


That is quite a few! This is not that unexpected seeing that we were now acting on a subset of the first ~10% of the samples (i.e. many of the samples in the file do not have the ALT alleles). But these "empty" lines come with a cost for storage and parsing, so for our downsampling we can drop these lines. This time, let's set the minimum AC count to 1, using the flag `-c1` (note the lower case `c` for the min AC filter, compared to captial `C` for the max AC filter)

In [26]:
%%time
!bcftools view -c1 ./input_data_temp/1kG_p3_chr1_first_200_samples.bcf -Ob -o ./input_data_temp/1kG_p3_chr1_first_200_samples_c1.bcf

CPU times: user 22.7 ms, sys: 22.8 ms, total: 45.4 ms
Wall time: 7.78 s


This really reduced the file size:

In [27]:
downsampled_bcf="./input_data_temp/1kG_p3_chr1_first_200_samples_c1.bcf"
file_size = os.path.getsize(downsampled_bcf)
print(f"{os.path.basename(downsampled_bcf)}\t{humanfriendly.format_size(file_size, binary=True)}")

1kG_p3_chr1_first_200_samples_c1.bcf	72.55 MiB


Sanity-check query to ensure that there are no AC=0 variants:

In [28]:
%%time
!bcftools view --no-header -C0 {downsampled_bcf} |wc -l

0
CPU times: user 4.54 ms, sys: 13.7 ms, total: 18.2 ms
Wall time: 796 ms


In [29]:
%%time
!bcftools view --no-header {downsampled_bcf} |wc -l

1326846
CPU times: user 16.1 ms, sys: 19.7 ms, total: 35.8 ms
Wall time: 4.3 s


And about 1 million variants in total. This level of downsampling is probably enough for now. Let's finish by converting the downsampled file to VCF.GZ so that we get the option to explore ingestion of BCF and VCF.GZ in the coming notebooks.

In [30]:
%%time
!bcftools view {downsampled_bcf} -Oz -o ./input_data_temp/1kG_p3_chr1_first_200_samples_c1.vcf.gz

CPU times: user 38.9 ms, sys: 29 ms, total: 67.9 ms
Wall time: 12.7 s


As it turns out, for this data at this degree of downsampling, there is not very big differences in file size between the BCF and VCF.GZ:

In [34]:
downsampled_vcf_gz = "./input_data_temp/1kG_p3_chr1_first_200_samples_c1.vcf.gz"
file_size = os.path.getsize(downsampled_vcf_gz)
print(f"{os.path.basename(downsampled_vcf_gz)}\t{humanfriendly.format_size(file_size, binary=True)}")
file_size = os.path.getsize(downsampled_bcf)
print(f"{os.path.basename(downsampled_bcf)}\t{humanfriendly.format_size(file_size, binary=True)}")

1kG_p3_chr1_first_200_samples_c1.vcf.gz	79.62 MiB
1kG_p3_chr1_first_200_samples_c1.bcf	72.55 MiB


Alright. With this, we now have downsampled dataset of genetic variants to use when exploring different tools for working with VCF files. We should probably still consider this a small dataset as far as population genomics is considered. But hopefully these 1 million variants and 200 samples is big enough to get processing times is seconds or minutes rather than micro- or milliseconds so that we can get a little feeling for their resource demands.